## Load env file


In [9]:
from dotenv import load_dotenv

load_dotenv()


True

## Initialize LLM


In [10]:
from crewai import LLM

llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.1)


## Create a tool


In [12]:
from crewai.tools import BaseTool


class ReplacejargonsTool(BaseTool):
    name: str = "Replace Jargons"
    description: str = "Replace jargons with specific words"

    def _run(self, email: str) -> str:
        """
        This tool replaces jargons with specific words
        """

        replace_jargons = {
            " :-) ": "smily face",
            "ping me": "reach out to me",
            "wip": "work in progress",
            "eta": "estimated time of arrival",
            "prx": "product requirements exchange",
            "tas": "technical assistance",
            "sds": "software development standards",
            "syncbot": "syncbot",
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replace_jargons.items():
            if jargon in email_lower:
                suggestions.append(f"consider replacing {jargon} -> {replacement}")
        return "\n".join(suggestions) if suggestions else "No jargons found"


jt = ReplacejargonsTool()

## Create a Agent and Task


In [13]:
from crewai import Agent, Task, Crew


def rewrite_email(email_text: str) -> str:
    email_agent = Agent(
        role="Email Agent",
        goal="Send an email to the CEO of the company",
        backstory="You are a helpful assistant that can sends professional emails",
        llm=llm,
        verbose=True,
        tools=[jt],
    )

    email_task = Task(
        description=f"""take the following email and rewrite it to professional version. 
    Original email: '''{email_text}'''
    """,
        agent=email_agent,
        expected_output="An email to the colleagues of the company, expanded the aberrations",
    )

    email_agent_crew = Crew(agents=[email_agent], tasks=[email_task], verbose=True)
    result = email_agent_crew.kickoff()
    return result

In [ ]:
# Provide your email text and run this cell
email_text = """
hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff left.
Maybe we can show what we have and say rest is WIP.
Let me know what u think. thanks
ping me if any blockers.

PRX and WIP are in the deck.
"""

print(rewrite_email(email_text))
